In [29]:
import numpy as np
import pandas as pd

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [30]:
ratings = pd.read_csv(
    "../data/ml-100k/u.data",
    sep="\t",
    names=["user_id", "movie_id", "rating", "timestamp"]
)

ratings.head()


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [31]:
np.random.seed(42)

test_size = 0.2
test_indices = np.random.choice(
    ratings.index,
    size=int(len(ratings) * test_size),
    replace=False
)

test_ratings = ratings.loc[test_indices]
train_ratings = ratings.drop(test_indices)

print("Train size:", len(train_ratings))
print("Test size:", len(test_ratings))


Train size: 80000
Test size: 20000


In [32]:
train_matrix = train_ratings.pivot(
    index="user_id",
    columns="movie_id",
    values="rating"
)

train_matrix.head()


movie_id,1,2,3,4,5,6,7,8,9,10,...,1668,1670,1671,1672,1673,1676,1678,1679,1680,1681
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.0,4.0,NaN,3.0,NaN,4.0,NaN,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
user_means = train_matrix.mean(axis=1)

train_centered = train_matrix.sub(user_means, axis=0)

train_filled = train_centered.fillna(0)


In [34]:
k = 20

svd = TruncatedSVD(n_components=k, random_state=42)
U = svd.fit_transform(train_filled.values)
VT = svd.components_
